In [ ]:
import pandas as pd

In [ ]:
import tensorflow_hub as hub

In [ ]:
from google.colab import drive


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
file_path='/content/gdrive/MyDrive/UrbanSoundFiles'

In [ ]:
import pandas as pd
metadata=pd.read_csv('/content/gdrive/MyDrive/UrbanSoundFiles/UrbanSound8K.csv')


In [ ]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

    return mfccs_scaled_features


In [2]:
pip install resampy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 4.3 MB/s eta 0:00:00


In [ ]:
import resampy

In [ ]:
import os
import numpy as np
from tqdm import tqdm
import librosa
import resampy

def features_extractor(file):
    audio, sample_rate = librosa.load(file, sr=None)
    target_sample_rate = 22050
    if sample_rate != target_sample_rate:
        audio = resampy.resample(audio, sample_rate, target_sample_rate)
        sample_rate = target_sample_rate
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    return mfccs_scaled_features



In [ ]:
extracted_features = []
for index_num, row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(file_path), 'fold' + str(row["fold"]) + '/', str(row["slice_file_name"]))
    final_class_labels = row["class"]
    data = features_extractor(file_name)
    extracted_features.append([data, final_class_labels])


In [ ]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-214.35951, 66.00463, -126.225266, -57.349586...",dog_bark
1,"[-420.23825, 103.904526, -47.55043, 55.606483,...",children_playing
2,"[-455.2568, 116.408104, -41.6107, 47.207104, 4...",children_playing
3,"[-409.9394, 96.08554, -29.919292, 47.638355, 6...",children_playing
4,"[-442.84656, 108.39454, -47.17804, 55.187874, ...",children_playing


In [ ]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [ ]:
y=np.array(pd.get_dummies(y))


In [ ]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=uint8)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [ ]:
num_labels=y.shape[1]


**DENSE** **MODEL**

In [ ]:
model=Sequential()
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5',
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/100
206/219 [===========================>..] - ETA: 0s - loss: 9.1004 - accuracy: 0.1224
Epoch 1: val_loss improved from inf to 2.28964, saving model to saved_models/audio_classification.hdf5
219/219 [==============================] - 2s 5ms/step - loss: 8.7486 - accuracy: 0.1227 - val_loss: 2.2896 - val_accuracy: 0.1208
Epoch 2/100
 28/219 [==>...........................] - ETA: 0s - loss: 2.6092 - accuracy: 0.1261

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


203/219 [==========================>...] - ETA: 0s - loss: 2.4866 - accuracy: 0.1235
Epoch 2: val_loss improved from 2.28964 to 2.28283, saving model to saved_models/audio_classification.hdf5
219/219 [==============================] - 1s 4ms/step - loss: 2.4786 - accuracy: 0.1246 - val_loss: 2.2828 - val_accuracy: 0.1128
Epoch 3/100
212/219 [============================>.] - ETA: 0s - loss: 2.3253 - accuracy: 0.1206
Epoch 3: val_loss improved from 2.28283 to 2.27172, saving model to saved_models/audio_classification.hdf5
219/219 [==============================] - 1s 4ms/step - loss: 2.3251 - accuracy: 0.1210 - val_loss: 2.2717 - val_accuracy: 0.1151
Epoch 4/100
206/219 [===========================>..] - ETA: 0s - loss: 2.2754 - accuracy: 0.1329
Epoch 4: val_loss improved from 2.27172 to 2.18946, saving model to saved_models/audio_classification.hdf5
219/219 [==============================] - 1s 4ms/step - loss: 2.2765 - accuracy: 0.1329 - val_loss: 2.1895 - val_accuracy: 0.1855
Epoch 5

In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.7704636454582214


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
num_samples = 6985
sequence_length = 1
num_features = 40
num_classes = 10

In [ ]:
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)


**GRU** **MODEL**

In [ ]:
model1=tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
    tf.keras.layers.GRU(32,return_sequences=True,activation='sigmoid'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GRU(64,return_sequences=True,activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GRU(num_labels,return_sequences=False,activation='relu'),
    tf.keras.layers.Dropout(0.2)
])
model1.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.02),
               metrics=['accuracy'])
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-5,
    verbose=1
)
model1.fit(X_train_reshaped, y_train, batch_size=num_batch_size, epochs=num_epochs,validation_split=0.2, callbacks=[lr_scheduler])

Epoch 1/100
175/175 [==============================] - 18s 75ms/step - loss: 6.3848 - accuracy: 0.1188 - val_loss: 4.3256 - val_accuracy: 0.1203 - lr: 0.0200
Epoch 2/100
175/175 [==============================] - 13s 73ms/step - loss: 5.4128 - accuracy: 0.1401 - val_loss: 2.8021 - val_accuracy: 0.1969 - lr: 0.0200
Epoch 3/100
175/175 [==============================] - 11s 63ms/step - loss: 5.2320 - accuracy: 0.1675 - val_loss: 2.8017 - val_accuracy: 0.1811 - lr: 0.0200
Epoch 4/100
175/175 [==============================] - 11s 61ms/step - loss: 5.1990 - accuracy: 0.1534 - val_loss: 2.8940 - val_accuracy: 0.1167 - lr: 0.0200
Epoch 5/100
175/175 [==============================] - 12s 67ms/step - loss: 5.2552 - accuracy: 0.1299 - val_loss: 2.8714 - val_accuracy: 0.1403 - lr: 0.0200
Epoch 6/100
174/175 [============================>.] - ETA: 0s - loss: 5.2542 - accuracy: 0.1487
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.009999999776482582.
175/175 [=============================

**MAIN** **MODEL**

In [ ]:
model2 = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GRU(128, return_sequences=True, activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GRU(64, return_sequences=False, activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_labels, activation='softmax')
])
model2.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
model2.fit(X_train_reshaped, y_train, batch_size=num_batch_size, epochs=num_epochs,validation_split=0.2, callbacks=[lr_scheduler])

Epoch 1/100
175/175 [==============================] - 11s 32ms/step - loss: 1.7100 - accuracy: 0.3803 - val_loss: 1.3218 - val_accuracy: 0.5211 - lr: 0.0010
Epoch 2/100
175/175 [==============================] - 7s 42ms/step - loss: 1.2436 - accuracy: 0.5653 - val_loss: 1.0703 - val_accuracy: 0.6263 - lr: 0.0010
Epoch 3/100
175/175 [==============================] - 6s 34ms/step - loss: 1.0341 - accuracy: 0.6385 - val_loss: 0.9381 - val_accuracy: 0.6779 - lr: 0.0010
Epoch 4/100
175/175 [==============================] - 7s 40ms/step - loss: 0.8965 - accuracy: 0.6981 - val_loss: 0.8315 - val_accuracy: 0.7130 - lr: 0.0010
Epoch 5/100
175/175 [==============================] - 5s 30ms/step - loss: 0.8008 - accuracy: 0.7332 - val_loss: 0.7178 - val_accuracy: 0.7545 - lr: 0.0010
Epoch 6/100
175/175 [==============================] - 6s 33ms/step - loss: 0.6948 - accuracy: 0.7657 - val_loss: 0.6972 - val_accuracy: 0.7759 - lr: 0.0010
Epoch 7/100
175/175 [==============================] - 6s